In [2]:
import numpy as np
import pandas as pd
import json
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.spatial.distance import cdist


In [3]:
COLOR_PALETTE = ['#d0e3f5',
                 '#712e67',
                 '#267592',
                 '#5fb12a',
                 '#fac800',
                 '#ff7917',
                 '#e23a34']

In [4]:
curves = ["curve_1", "curve_2"]

def load_curve(filename):
    return np.load('pointClouds/'  + filename + '.npy')

for el in curves:
    print(len(load_curve(el)))

132
101


# Communities partition

In [8]:
COMMS = {}
with open('communities.json') as fh:
    PARTITIONS = json.load(fh)
    for curve in curves:
        partition = PARTITIONS[curve]
        COMMS[curve] = {c:[] for c in set(partition)}
        for vertex, c in enumerate(partition):
            COMMS[curve][c].append(vertex)

In [9]:
def community_adjacency(curve, scope='plot'):
    """
    Outputs the community adjacency matrix of the curve.
    Inputs:
    - curve: a string indicating the filename of the curve
    - scope: a string. If scope == 'plot', the adjacency matrix has 
    entries equal to community labels. If scope is any other string, 
    entries of the matrix are zeros and ones. 
    """
    
    cs = COMMS[curve]
    n = np.sum([len(cs[k]) for k in cs.keys()])
    matrix = np.zeros((n,n))

    color = 1
    for group in cs.keys():
        if len(cs[group]) >1:
            for i in cs[group]:
                for j in cs[group]:
                    if scope == 'plot':
                        matrix[i,j] = color
                    else:
                        matrix[i,j] = 1
            color = color+1

    return(matrix) 


def plot_chain_with_communities(filename):
    
    """
    Plots the curve stored as filename with points colour-coded according
    to community memebership
    """
    
    community = COMMS[filename]
    curve = load_curve(filename)
    df = pd.DataFrame.from_records(curve, columns=['X', 'Y', 'Z'])
    
    nodes = communities_color_code(community)
    df["color"] = nodes    
        
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
    x=df['X'], y=df['Y'], z=df['Z'], 
    name = 'Curve',    
    marker=dict(
        size=8,
        color = df["color"],
        colorscale=COLOR_PALETTE,
        line=dict(width=4, 
                  color='DarkSlateGrey')
    ),
        
    line=dict(
        width=8,
        color = df["color"],
        colorscale=COLOR_PALETTE
    )),
    )
    fig.update_layout(scene=dict(xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False,),
    yaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
               zaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
        camera=dict(
            up=dict(
                x=-10,
                y=0,
                z=30
            ),
            eye=dict(
                x=0.9,
                y=0.9,
                z=1.3,
            )
        )))
                  
    fig.update_layout(scene = dict(
                    xaxis_title=' ',
                    yaxis_title=' ',
                    zaxis_title=' '))
    fig.update_traces(hoverinfo="text",hovertemplate=nodes)

    return fig


## Visualising spatial modules given by communities

To improve visualisation, points in communities formed by single points are considered as unclassified and are all coloured in light blue

In [10]:
def communities_color_code(comms):
    """
    Assigns a label to each community whith at least two memebers.
    """    
    
    n = np.sum([len(comms[k]) for k in comms.keys()])
    nodes = n*[0]
    color = 1
    for k in comms.keys():
        if len(comms[k])>1:
            for j in comms[k]:
                nodes[j] = color
            color = color+1
    return nodes

In [11]:
fig = plot_chain_with_communities('curve_1')
fig.show()

In [12]:
M = community_adjacency('curve_1')
px.imshow(M, color_continuous_scale= COLOR_PALETTE)

### Pairwise distances alone fail to recover the structural features of the curve

In [13]:
a = load_curve('curve_1')
M = cdist(a,a)
px.imshow(M, color_continuous_scale=  [(0,'#d0e3f5'), (1,"blue")] )

In [14]:
fig = plot_chain_with_communities('curve_2')
fig.show()

In [15]:
M = community_adjacency('curve_2')
px.imshow(M, color_continuous_scale= COLOR_PALETTE)

### Pairwise distances alone fail to recover the structural features of the curve

In [16]:
a = load_curve('curve_2')
M = cdist(a,a)
px.imshow(M, color_continuous_scale=  [(0,'#d0e3f5'), (1,"blue")] )

# Node centrality

In [17]:
def load_node_centrality(filename):
    """
    Returns the centrality vector for filename. Note: centrality vectors are saved
    by pipeline without terminal zeros, so we are appending zeros to match the length of
    the curve. 
    
    """
    cc = pd.read_csv('nodeCents/' + filename + '.tsv', sep="\t", header=None).values
    for i in range(len(cc),len(load_curve(filename))):
        cc = np.append(cc,0)
    return cc


def plot_chain_with_centrality(filename):
    
    """
    Plots the curve stored as filename with points colour-coded according
    to centrality value
    """
        
    
    community = COMMS[filename]
    curve = load_curve(filename)
    df = pd.DataFrame.from_records(curve, columns=['X', 'Y', 'Z'])
    
    nodes = load_node_centrality(filename)
    df["color"] = nodes    
        
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
    x=df['X'], y=df['Y'], z=df['Z'], 
    name = 'Curve',    
    marker=dict(
        size=8,
        color = df["color"],
        line=dict(width=4, 
                  color='DarkSlateGrey')
    ),
        
    line=dict(
        width=8,
        color = df["color"],
    )),
    )
    fig.update_layout(scene=dict(xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False,),
    yaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
               zaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
        camera=dict(
            up=dict(
                x=-10,
                y=0,
                z=30
            ),
            eye=dict(
                x=0.9,
                y=0.9,
                z=1.3,
            )
        )))
                  
    fig.update_layout(scene = dict(
                    xaxis_title=' ',
                    yaxis_title=' ',
                    zaxis_title=' '))

    return fig

In [18]:
fig = plot_chain_with_centrality('curve_2')
fig.show()

In [19]:
fig = plot_chain_with_centrality('curve_1')
fig.show()

Alternative plot of node centrality 

In [20]:
filename = "curve_1"
curve = load_curve(filename)
centrality = load_node_centrality(filename)
df = pd.DataFrame.from_records(curve, columns=['X', 'Y', 'Z'])

n_nodes = df.shape[0] 
positions = [i for i in range(n_nodes)]


In [36]:
fig = px.scatter(x=positions, y=[0] * n_nodes, color=centrality,
                labels = dict(x = "Vertex position", y = "PH centrality", color = "PH centrality"))
fig.add_scatter(x=positions, y=centrality, mode = "markers + lines", showlegend = False)
fig.update_coloraxes(showscale=False)
fig.update_layout(
    width=1000,
    height=500,
    font=dict(
    size=18
    ))
fig.write_image("PH_centrality.pdf")
fig.show()


In [41]:
def plot_chain_with_centrality_binary(filename, cutoff):
    
    """
    Plots the curve stored as filename with points colour-coded according
    to centrality value.
    Points with centrality value below cutoff will be plotted in yellow.
    Points with centrality value above the cutoff will be plotted in red. 
    """
        
    
    community = COMMS[filename]
    curve = load_curve(filename)
    df = pd.DataFrame.from_records(curve, columns=['X', 'Y', 'Z'])
    
    nodes = load_node_centrality(filename)
    nodes = [int(x) for x in nodes > cutoff]

    # label: color
    color_dict = {
        0: "#faa445",
        1: "#00d8dc",
    }

    # fillna is just-in-case for new/other labels
    colors = [color_dict[i] for i in nodes]
    

        
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
    x=df['X'], y=df['Y'], z=df['Z'], 
    name = 'Curve', 
    marker=dict(
        size=8,
        color = colors,
        line=dict(width=4, 
                  color='DarkSlateGrey')
    ),

    line=dict(
        width=8,
        color = colors,
    )
    ),
    )
    fig.update_layout(scene=dict(xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False,),
    yaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
               zaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
        camera=dict(
            up=dict(
                x=-10,
                y=0,
                z=30
            ),
            eye=dict(
                x=0.9,
                y=0.9,
                z=1.3,
            )
        )))
                  
    fig.update_layout(scene = dict(
                    xaxis_title=' ',
                    yaxis_title=' ',
                    zaxis_title=' '))

    return fig

In [42]:
fig = plot_chain_with_centrality_binary('curve_1', 0.0087)
fig.show()